# Double Factorization

Bloqs for the double-factorized chemistry Hamiltonian in second quantization.

Recall that for the single factorized Hamiltonian we have
$$
H = \sum_{pq}T'_{pq} a_p^\dagger a_q + \frac{1}{2} \sum_l \left(\sum_{pq} 
W_{pq}^{(l)} a_p^\dagger a_q\right)^2.
$$
One arrives at the double factorized (DF) Hamiltonian by further factorizing the
$W_{pq}^{(l)}$ terms as
$$
W^{(l)}_{pq} = \sum_{k} U^{(l)}_{pk} f_k^{(l)} U^{(l)*}_{qk},
$$
so that
$$
H = \sum_{pq}T'_{pq} a_p^\dagger a_q + \frac{1}{2} \sum_l U^{(l)}\left(\sum_{k}^{\Xi^{(l)}} 
f_k^{(l)} n_k\right)^2 U^{(l)\dagger}
$$
where $\Xi^{(l)} $ is the rank of second factorization. In principle one can
truncate the second factorization to reduce the amount of information required
to specify the Hamiltonian. However this somewhat complicates the implementation
and it is more convenient to fix the rank of the second factorization for all
$l$ terms. Nevertheless the authors of Ref[1] chose to do it the hard way.

In [1]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran.drawing import show_bloq
from typing import *
import numpy as np

## `DoubleFactorization`
Block encoding of double factorization Hamiltonian.

Implements Fig. 15 in the reference.

#### Parameters
 - `num_spin_orb`: The number of spin orbitals. Typically called N.
 - `num_aux`: Dimension of auxiliary index for double factorized Hamiltonian. Called L in Ref[1].
 - `num_bits_state_prep`: The number of bits of precision for coherent alias sampling. Called aleph (fancy N) in Ref[1].
 - `num_bits_rot`: Number of bits of precision for rotations for amplitude amplification in uniform state preparation. Called b_r in Ref[1].
 - `qroam_k_factor`: QROAM blocking factor for data prepared over l (auxiliary) index. Defaults to 1 (i.e. use QROM). 

#### Registers
 - `l`: register to store L values for auxiliary index.
 - `succ_pq`: flag for success of this state preparation.
 - `succ_l`: flag for success of this state preparation. 

Refererences:
    [Even More Efficient Quantum Computations of Chemistry Through Tensor
        hypercontraction](https://journals.aps.org/prxquantum/pdf/10.1103/prxquantum.2.030305)

In [2]:
from qualtran.bloqs.chemistry.double_factorization import DoubleFactorization

bloq = DoubleFactorization(10, 20, 8)
show_bloq(bloq.decompose_bloq())

ValueError: Invalid Connection OutputIndexedData(num_aux=20, num_spin_orb=10, num_xi=8, num_bits_rot_aa=8, ko=1, adjoint=False)<6>.rot -> BlockEncodeChebyshevPolynomial(block_encoding=DoubleFactorizationOneBody(num_aux=20, num_spin_orb=10, num_xi=8, num_bits_state_prep=8, num_bits_rot_aa=8, num_bits_rot=24, adjoint=False), order=2)<8>.rot: shape mismatch: 8 != 1

In [ ]:
from qualtran.resource_counting import get_bloq_counts_graph
from qualtran.drawing import show_counts_graph, show_counts_sigma
from qualtran.bloqs.util_bloqs import Split, Join, Allocate, Free

def generalize(bloq):
    if isinstance(bloq, (Split, Join, Allocate, Free)):
        return None
    return bloq

graph, sigma = get_bloq_counts_graph(DoubleFactorization(10, 50, 10//2, num_bits_state_prep=12, num_bits_rot_aa_inner=7, num_bits_rot_aa_outer=1, num_bits_rot=12), generalizer=generalize)
show_counts_graph(graph)

In [ ]:
# outer prep (correct)
(116*2 + 176 + 60) / 4

In [ ]:
# in-data off by 6
print((60+180) / 4)

In [ ]:
# state prep reg 2 (off by 1 for qrom)
2 * (128  + 460) / 4

In [ ]:
2 * ( 2 * 204)/4 + 294 # off by 1

In [ ]:
2*(428 + 752 + 128 + 428 + 2*20) / 4